In [1]:
import pandas as pd
import numpy as np
import cx_Oracle
from config import oracle_wfm_key
from config import oracle_ods_key
from sqlalchemy.types import String
from tqdm import tqdm
from datetime import datetime
import datetime as dt
import time
import databases as db

In [2]:
ods = db.oracle('usrocu_support',oracle_ods_key)   
query = f'''SELECT DISTINCT
                DATA_DATE AS call_date,
                STATS.INBOUND_HANDLE,
                STATS.ABANDONED_CALLS,
                STATS.QUEUETIME,
                STATS.ANSWERED_CALLS,
                STATS.OFFERED_CALLS,
                STATS.TRANSFERS,
                STATS.IN_SL,
                service_c as service_name
            FROM
                cco_objects.mv_calls_per_agent_per_service stats
                LEFT JOIN uip.C_SERVICE sc ON sc.service_id = stats.service_id
                LEFT JOIN uip.r_service_to_service_group ssg ON ssg.service_id = stats.service_id
                LEFT JOIN uip.r_service_group sg ON ssg.service_group_id = sg.service_group_id
            WHERE
                stats.data_date >= TO_DATE('2020-01-01', 'YYYY-MM-DD')
'''


In [3]:
df = pd.read_sql(query, ods)
df.head(3)

,call_date,inbound_handle,abandoned_calls,queuetime,answered_calls,offered_calls,transfers,in_sl,service_name
0,2020-10-08,998,0,18,4,4,2,4,Bend_Customer_Care_Sales
1,2020-10-08,2193,0,317,2,2,0,1,Wireline_Repair_Video_Res
2,2020-02-25,8522,0,2504,13,13,0,8,Cable_Repair_Data_Res


In [4]:
df['call_date'] = pd.to_datetime(df['call_date'])  # Convert 'call_date' column to datetime if needed

# Set 'call_date' as the index
df.set_index('call_date', inplace=True)

# Group by 'service_name' and resample to monthly frequency
df_resampled = df.groupby('service_name').resample('MS').sum()

# Reset the index to make 'service_name' and 'call_date' regular columns again
df_resampled.reset_index(inplace=True)

df_resampled.head()

,service_name,call_date,inbound_handle,abandoned_calls,queuetime,answered_calls,offered_calls,transfers,in_sl
0,ABS_Repair_Bus_Sales_Support,2020-01-01,5576,1,748,10,11,3,9
1,ABS_Repair_Bus_Sales_Support,2020-02-01,4272,0,282,10,10,1,8
2,ABS_Repair_Bus_Sales_Support,2020-03-01,21385,1,1885,42,44,18,36
3,ABS_Repair_Bus_Sales_Support,2020-04-01,8995,0,206,14,14,4,13
4,ABS_Repair_Bus_Sales_Support,2020-05-01,14233,2,236,13,15,2,13


In [5]:
uip_df = df_resampled[['service_name','call_date','offered_calls','in_sl','inbound_handle','queuetime','abandoned_calls','transfers']]
uip_df['sl%'] = df_resampled['in_sl'] / df_resampled['offered_calls']
uip_df['abandon%'] = df_resampled['abandoned_calls'] / df_resampled['offered_calls']
uip_df['asa'] = df_resampled['queuetime'] / df_resampled['offered_calls']
uip_df['aht'] = df_resampled['inbound_handle'] / df_resampled['offered_calls']
uip_df.fillna(0, inplace=True)
uip_df.head()

,service_name,call_date,offered_calls,in_sl,inbound_handle,queuetime,abandoned_calls,transfers,sl%,abandon%,asa,aht
0,ABS_Repair_Bus_Sales_Support,2020-01-01,11,9,5576,748,1,3,0.818182,0.090909,68.000000,506.909091
1,ABS_Repair_Bus_Sales_Support,2020-02-01,10,8,4272,282,0,1,0.800000,0.000000,28.200000,427.200000
2,ABS_Repair_Bus_Sales_Support,2020-03-01,44,36,21385,1885,1,18,0.818182,0.022727,42.840909,486.022727
3,ABS_Repair_Bus_Sales_Support,2020-04-01,14,13,8995,206,0,4,0.928571,0.000000,14.714286,642.500000
4,ABS_Repair_Bus_Sales_Support,2020-05-01,15,13,14233,236,2,2,0.866667,0.133333,15.733333,948.866667


In [6]:
ods = db.oracle('cco_wfm',oracle_wfm_key)
occ = f'''SELECT * FROM "CCO_WFM"."Occupancy_Regressors"'''
df1 = pd.read_sql(occ, ods)

In [7]:
occupancy = df1[['Date','Workgroup_Set','Occupancy']]
occupancy.columns = ['call_date','workgroup_set','occupancy']
occupancy['call_date'] = pd.to_datetime(occupancy['call_date'])
occupancy = occupancy.groupby(['call_date', 'workgroup_set'])['occupancy'].max().reset_index()
occupancy.head()

C:\Users\usrocu\AppData\Local\Temp\ipykernel_11756\2107282427.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  occupancy['call_date'] = pd.to_datetime(occupancy['call_date'])


,call_date,workgroup_set,occupancy
0,2021-08-01,APC,0.065472
1,2021-08-01,COAST,0.171272
2,2021-08-01,Cable_Consumer_Sales,0.632041
3,2021-08-01,Cable_Consumer_Specialist,0.530829
4,2021-08-01,Cable_Seniors_CCO_Support,0.471298


In [8]:
from uip_campaign_dict import campaign
uip_df['workgroup_set'] = ""
uip_df['workgroup_set'] = uip_df['service_name'].map(campaign)
uip_df.head()

,service_name,call_date,offered_calls,in_sl,inbound_handle,queuetime,abandoned_calls,transfers,sl%,abandon%,asa,aht,workgroup_set
0,ABS_Repair_Bus_Sales_Support,2020-01-01,11,9,5576,748,1,3,0.818182,0.090909,68.000000,506.909091,Repair_ABS
1,ABS_Repair_Bus_Sales_Support,2020-02-01,10,8,4272,282,0,1,0.800000,0.000000,28.200000,427.200000,Repair_ABS
2,ABS_Repair_Bus_Sales_Support,2020-03-01,44,36,21385,1885,1,18,0.818182,0.022727,42.840909,486.022727,Repair_ABS
3,ABS_Repair_Bus_Sales_Support,2020-04-01,14,13,8995,206,0,4,0.928571,0.000000,14.714286,642.500000,Repair_ABS
4,ABS_Repair_Bus_Sales_Support,2020-05-01,15,13,14233,236,2,2,0.866667,0.133333,15.733333,948.866667,Repair_ABS


In [9]:
data_df = uip_df.merge(occupancy, on=['call_date','workgroup_set'], how='inner')
data_df
data_df.head()

,service_name,call_date,offered_calls,in_sl,inbound_handle,queuetime,abandoned_calls,transfers,sl%,abandon%,asa,aht,workgroup_set,occupancy
0,ABS_Repair_Bus_Sales_Support,2021-08-01,15,12,15210,640,0,4,0.800000,0.000000,42.666667,1014.000000,Repair_ABS,0.482479
1,ABS_Repair_Cable_Managed_IP,2021-08-01,70,60,71456,1545,1,16,0.857143,0.014286,22.071429,1020.800000,Repair_ABS,0.482479
2,ABS_Repair_Carrier_Circuit,2021-08-01,4,4,3122,6,0,0,1.000000,0.000000,1.500000,780.500000,Repair_ABS,0.482479
3,ABS_Repair_Legacy_Fiber,2021-08-01,58,42,52451,3456,3,9,0.724138,0.051724,59.586207,904.327586,Repair_ABS,0.482479
4,ABS_Repair_Level_II_Support,2021-08-01,487,358,485415,15487,14,3,0.735113,0.028747,31.800821,996.745380,Repair_ABS,0.482479


In [10]:
# finish cleaning data_df
# Step 1: Drop rows with missing values in the 'occupancy' column
data_df.dropna(subset=['occupancy'], inplace=True)

# Step 2: Drop rows with negative values in the 'aht' column
data_df = data_df[data_df['aht'] >= 0]

# Step 3: Replace infinite values in the 'asa' column with NaN
data_df.replace([np.inf, -np.inf], np.nan, inplace=True)

# Step 4: Replace infinite values in the 'occupancy' column with NaN
data_df['occupancy'].replace([np.inf, -np.inf], np.nan, inplace=True)

# Step 5: Impute missing values in 'asa' and 'occupancy' columns using mean imputation
data_df['asa'].fillna(data_df['asa'].mean(), inplace=True)
data_df['occupancy'].fillna(data_df['occupancy'].mean(), inplace=True)

In [11]:
data_df.describe()

,offered_calls,in_sl,inbound_handle,queuetime,abandoned_calls,transfers,sl%,abandon%,asa,aht,occupancy
count,4663.000000,4663.000000,4.663000e+03,4.663000e+03,4663.000000,4663.000000,4663.000000,4663.000000,4663.000000,4663.000000,4663.000000
mean,1573.049539,1051.752734,9.108071e+05,2.526534e+05,134.521124,278.745228,0.479550,0.093386,69.876829,390.875250,0.368576
std,4341.615446,2865.458444,2.607514e+06,1.341251e+06,551.309126,1102.072072,0.388257,0.208596,394.184466,396.109625,0.130421
min,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.036807
25%,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.269131
50%,72.000000,43.000000,3.357100e+04,2.539000e+03,3.000000,3.000000,0.632381,0.023256,26.383143,363.861338,0.344152
75%,757.500000,535.000000,3.871875e+05,4.775100e+04,43.000000,44.000000,0.824376,0.082491,81.629113,666.284921,0.419467
max,32742.000000,29236.000000,2.434595e+07,2.843374e+07,11938.000000,13375.000000,1.000000,1.000000,23912.052478,3288.000000,1.056567


In [12]:
# Move 'workgroup_set' to the first column using iloc
workgroup_set_col = data_df.pop('workgroup_set')
data_df.insert(0, 'workgroup_set', workgroup_set_col)
data_df.head()

,workgroup_set,service_name,call_date,offered_calls,in_sl,inbound_handle,queuetime,abandoned_calls,transfers,sl%,abandon%,asa,aht,occupancy
0,Repair_ABS,ABS_Repair_Bus_Sales_Support,2021-08-01,15,12,15210,640,0,4,0.800000,0.000000,42.666667,1014.000000,0.482479
1,Repair_ABS,ABS_Repair_Cable_Managed_IP,2021-08-01,70,60,71456,1545,1,16,0.857143,0.014286,22.071429,1020.800000,0.482479
2,Repair_ABS,ABS_Repair_Carrier_Circuit,2021-08-01,4,4,3122,6,0,0,1.000000,0.000000,1.500000,780.500000,0.482479
3,Repair_ABS,ABS_Repair_Legacy_Fiber,2021-08-01,58,42,52451,3456,3,9,0.724138,0.051724,59.586207,904.327586,0.482479
4,Repair_ABS,ABS_Repair_Level_II_Support,2021-08-01,487,358,485415,15487,14,3,0.735113,0.028747,31.800821,996.745380,0.482479


In [13]:
ods = db.oracle('cco_wfm',oracle_wfm_key)
employees = f'select * from "CCO_WFM"."Employee_Table"'
df2 = pd.read_sql(employees, ods)
df2.head()

,Employee,usr,Hire_Date,Termination_Date,Organization,City,State,Country
0,"Atkinson, Christie",usrbqu,2016-11-21 06:00:00,NaT,US,Bend,OR,United States
1,"Auderer, Tasha",usruay,2016-11-21 06:00:00,2017-03-25 04:59:00,CRA III,Madison,WI,None
2,"Auguste, Hygina",usrlsq,2021-06-07 05:00:00,2021-08-07 04:59:00,Resi Wireline Repair,Vieux Fort,None,Saint Lucia
3,"Auguste, Kdan",usrugk,2021-01-25 06:00:00,2021-02-25 05:59:00,Resi Cable Repair,Montego Bay,None,Jamaica
4,"Auguste, Sherneige",SAUGU1,2023-06-20 05:00:00,2023-07-27 04:59:00,ITEL Wireline Sales,None,None,None


In [14]:
from organizations_dict import organizations_dict
df2['workgroup_set'] = ""
df2['workgroup_set'] = df2['Organization'].map(organizations_dict)
df2.head()

,Employee,usr,Hire_Date,Termination_Date,Organization,City,State,Country,workgroup_set
0,"Atkinson, Christie",usrbqu,2016-11-21 06:00:00,NaT,US,Bend,OR,United States,Other
1,"Auderer, Tasha",usruay,2016-11-21 06:00:00,2017-03-25 04:59:00,CRA III,Madison,WI,None,Repair_Wireline_Residential
2,"Auguste, Hygina",usrlsq,2021-06-07 05:00:00,2021-08-07 04:59:00,Resi Wireline Repair,Vieux Fort,None,Saint Lucia,Repair_Wireline_Residential
3,"Auguste, Kdan",usrugk,2021-01-25 06:00:00,2021-02-25 05:59:00,Resi Cable Repair,Montego Bay,None,Jamaica,Repair_Cable_Residential
4,"Auguste, Sherneige",SAUGU1,2023-06-20 05:00:00,2023-07-27 04:59:00,ITEL Wireline Sales,None,None,None,Wireline_Consumer_Sales


In [15]:
df2 = df2[df2['Termination_Date'] >= '2020-01-01']
df2['Termination_Month'] = df2['Termination_Date'].dt.to_period('M')
result = df2.groupby(['workgroup_set', 'Termination_Month']).size().reset_index(name='Count')
result.head()

,workgroup_set,Termination_Month,Count
0,APC,2020-01,2
1,APC,2020-08,2
2,APC,2021-04,2
3,APC,2021-05,1
4,APC,2021-08,1


In [16]:
# Step 1: Format Termination_Month as 'YYYY-MM-DD'
result['call_date'] = result['Termination_Month'].dt.to_timestamp()
result = result.drop('Termination_Month', axis=1)
result['attrition'] = result['Count']
result = result.drop('Count', axis=1)
# Step 2: Merge result with data_df
data_df = data_df.merge(result, on=['workgroup_set', 'call_date'], how='left')
data_df['attrition'].fillna(0, inplace=True)
data_df.head()

,workgroup_set,service_name,call_date,offered_calls,in_sl,inbound_handle,queuetime,abandoned_calls,transfers,sl%,abandon%,asa,aht,occupancy,attrition
0,Repair_ABS,ABS_Repair_Bus_Sales_Support,2021-08-01,15,12,15210,640,0,4,0.800000,0.000000,42.666667,1014.000000,0.482479,0.0
1,Repair_ABS,ABS_Repair_Cable_Managed_IP,2021-08-01,70,60,71456,1545,1,16,0.857143,0.014286,22.071429,1020.800000,0.482479,0.0
2,Repair_ABS,ABS_Repair_Carrier_Circuit,2021-08-01,4,4,3122,6,0,0,1.000000,0.000000,1.500000,780.500000,0.482479,0.0
3,Repair_ABS,ABS_Repair_Legacy_Fiber,2021-08-01,58,42,52451,3456,3,9,0.724138,0.051724,59.586207,904.327586,0.482479,0.0
4,Repair_ABS,ABS_Repair_Level_II_Support,2021-08-01,487,358,485415,15487,14,3,0.735113,0.028747,31.800821,996.745380,0.482479,0.0


In [17]:
data_df.describe()

,offered_calls,in_sl,inbound_handle,queuetime,abandoned_calls,transfers,sl%,abandon%,asa,aht,occupancy,attrition
count,4663.000000,4663.000000,4.663000e+03,4.663000e+03,4663.000000,4663.000000,4663.000000,4663.000000,4663.000000,4663.000000,4663.000000,4663.000000
mean,1573.049539,1051.752734,9.108071e+05,2.526534e+05,134.521124,278.745228,0.479550,0.093386,69.876829,390.875250,0.368576,4.387733
std,4341.615446,2865.458444,2.607514e+06,1.341251e+06,551.309126,1102.072072,0.388257,0.208596,394.184466,396.109625,0.130421,9.533217
min,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.036807,0.000000
25%,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.269131,1.000000
50%,72.000000,43.000000,3.357100e+04,2.539000e+03,3.000000,3.000000,0.632381,0.023256,26.383143,363.861338,0.344152,2.000000
75%,757.500000,535.000000,3.871875e+05,4.775100e+04,43.000000,44.000000,0.824376,0.082491,81.629113,666.284921,0.419467,4.000000
max,32742.000000,29236.000000,2.434595e+07,2.843374e+07,11938.000000,13375.000000,1.000000,1.000000,23912.052478,3288.000000,1.056567,49.000000


In [18]:
start = data_df['call_date'].min()
start = start.date()
start_date = f'{start}'
end = data_df['call_date'].max()
end = end.date()
end_date = f'{end}'

In [19]:
select = f'''SELECT DISTINCT
                    "A1"."date" "call_date",
                    "A1"."WORKGROUP",
                    "A1"."PAID",
                    "A1"."PHONE",
                    "A1"."ABSENCE",
                    "A1"."EXCEPTIONS",
                    "A1"."OFFPHONE"
 
            FROM 
                "Verint_Compliance" "A1"
            WHERE "A1"."date" >= TO_DATE('{start_date}','YYYY-MM-DD')
                AND "A1"."date" < TO_DATE('{end_date}','YYYY-MM-DD')'''
    
shrink = pd.read_sql(select, ods)

In [20]:
# Step 1: Convert 'date' column to datetime if it's not already
shrink['call_date'] = pd.to_datetime(shrink['call_date'])

# Step 2: Set 'date' column as the index
shrink.set_index('call_date', inplace=True)
# Step 3: Group the data by 'workgroup' and 'team'
grouped_data = shrink.groupby(['workgroup'])

# Step 4: Resample each group to millisecond frequency
shrink_df = grouped_data.resample('MS').sum()

# Step 5: Reset the index to turn 'workgroup', 'team', and 'date' into columns again
shrink_df.reset_index(inplace=True)

# Step 6: Fill any missing values with 0, in case there are no data points for a specific timestamp
shrink_df.fillna(0, inplace=True)

# Step 7: Display the resampled DataFrame
shrink_df.head()

,workgroup,call_date,paid,phone,absence,exceptions,offphone
0,<Default>,2021-08-01,0.0,0.0,0.0,0.0,0.0
1,<Default>,2021-09-01,0.0,0.0,16.0,0.0,0.0
2,<Default>,2021-10-01,0.0,0.0,0.0,0.0,0.0
3,<Default>,2021-11-01,0.0,0.0,0.0,0.0,0.0
4,<Default>,2021-12-01,0.0,0.0,0.0,0.0,0.0


In [21]:
shrink_df.loc[(shrink_df['workgroup'].str.contains('ITEL Cable Sales'))]['phone'].sum()

83780.84

In [22]:
from wg_campaign_dict import workgroups
shrink_df['workgroup_set'] = ""
shrink_df['workgroup_set'] = shrink_df['workgroup'].map(workgroups)
shrink_df.drop(columns='workgroup', inplace=True)
shrink_df.head()

,call_date,paid,phone,absence,exceptions,offphone,workgroup_set
0,2021-08-01,0.0,0.0,0.0,0.0,0.0,Other
1,2021-09-01,0.0,0.0,16.0,0.0,0.0,Other
2,2021-10-01,0.0,0.0,0.0,0.0,0.0,Other
3,2021-11-01,0.0,0.0,0.0,0.0,0.0,Other
4,2021-12-01,0.0,0.0,0.0,0.0,0.0,Other


In [23]:
shrink_df.loc[(shrink_df['workgroup_set'] == 'Cable_Consumer_Sales')]['phone'].nunique()

101

In [24]:
table = data_df.merge(shrink_df, on=['call_date','workgroup_set'], how='inner')
table.fillna(0, inplace=True)
table.head()

,workgroup_set,service_name,call_date,offered_calls,in_sl,inbound_handle,queuetime,abandoned_calls,transfers,sl%,abandon%,asa,aht,occupancy,attrition,paid,phone,absence,exceptions,offphone
0,Repair_ABS,ABS_Repair_Bus_Sales_Support,2021-08-01,15,12,15210,640,0,4,0.800000,0.000000,42.666667,1014.000000,0.482479,0.0,2879.1,2303.73,469.766665,179.71,463.75
1,Repair_ABS,ABS_Repair_Cable_Managed_IP,2021-08-01,70,60,71456,1545,1,16,0.857143,0.014286,22.071429,1020.800000,0.482479,0.0,2879.1,2303.73,469.766665,179.71,463.75
2,Repair_ABS,ABS_Repair_Carrier_Circuit,2021-08-01,4,4,3122,6,0,0,1.000000,0.000000,1.500000,780.500000,0.482479,0.0,2879.1,2303.73,469.766665,179.71,463.75
3,Repair_ABS,ABS_Repair_Legacy_Fiber,2021-08-01,58,42,52451,3456,3,9,0.724138,0.051724,59.586207,904.327586,0.482479,0.0,2879.1,2303.73,469.766665,179.71,463.75
4,Repair_ABS,ABS_Repair_Level_II_Support,2021-08-01,487,358,485415,15487,14,3,0.735113,0.028747,31.800821,996.745380,0.482479,0.0,2879.1,2303.73,469.766665,179.71,463.75


In [25]:
table.loc[(table['workgroup_set'] == 'Cable_Consumer_Sales')]['phone'].nunique()

101

In [26]:
table.describe()

,offered_calls,in_sl,inbound_handle,queuetime,abandoned_calls,transfers,sl%,abandon%,asa,aht,occupancy,attrition,paid,phone,absence,exceptions,offphone
count,71197.000000,71197.000000,7.119700e+04,7.119700e+04,71197.000000,71197.000000,71197.000000,71197.000000,71197.000000,71197.000000,71197.000000,71197.000000,71197.000000,71197.000000,71197.000000,71197.000000,71197.000000
mean,614.859320,439.903999,3.377813e+05,8.606265e+04,53.660660,89.903929,0.383854,0.109003,59.437297,279.158228,0.317034,5.945560,649.702110,418.874817,79.818559,98.032276,220.230396
std,2335.243412,1624.094484,1.412591e+06,7.779744e+05,309.303707,561.571038,0.408648,0.252525,493.958867,379.868345,0.083464,11.684014,813.432797,684.500289,105.384394,197.504295,362.032887
min,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.046690,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.262217,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,5.000000,0.000000,0.000000e+00,1.800000e+01,0.000000,0.000000,0.000000,0.000000,1.805195,0.000000,0.295866,3.000000,310.360000,0.000000,46.750000,0.410000,68.500000
75%,262.000000,175.000000,1.273950e+05,1.013700e+04,16.000000,14.000000,0.807692,0.071429,54.933571,550.000000,0.350670,4.000000,1043.310000,682.000000,130.333333,70.120000,262.690000
max,32742.000000,29236.000000,2.434595e+07,2.843374e+07,11938.000000,13375.000000,1.000000,1.000000,23912.052478,3288.000000,1.056567,49.000000,8070.700000,5609.150000,1712.266376,2372.300000,2248.160000


In [27]:
ods = db.oracle('cco_wfm',oracle_wfm_key)
drop = f'''DROP TABLE "CCO_WFM"."Staffing_Analysis_Regressors"'''
with ods as conn:
    conn.execute(drop)

cols = table.dtypes[table.dtypes=='object'].index
type_mapping = {col : String(100) for col in cols }

def chunker(seq, size):
    return (seq.iloc[pos:pos + size] for pos in range(0, len(seq), size))

def insert_with_progress(table):
    chunksize = int(len(table) / 10)
    with tqdm(total=len(table)) as pbar:
        for i, cdf in enumerate(chunker(table, chunksize)):
            replace = "replace" if i == 0 else "append"
            cdf.to_sql('Staffing_Analysis_Regressors',con=ods,schema='CCO_WFM',if_exists='append',index=False,dtype=type_mapping)
            pbar.update(chunksize)
            tqdm._instances.clear()

ods = db.oracle('cco_wfm',oracle_wfm_key)
insert_with_progress(table)

78309it [00:04, 16067.14it/s]                           


In [28]:
table['workgroup_set'].unique()

array(['Repair_ABS', 'Other', 'Cable_Consumer_Sales',
       'Cable_Consumer_Specialist', 'Repair_RPC', 'FST_Dispatch',
       'Financial_Services_Business', 'Financial_Services_Residential',
       'Repair_Cable_Business', 'Repair_Cable_Residential',
       'Repair_Continuum', 'Continuum_Sales', 'Repair_Wireline_Business',
       'APC', 'Commercial_Billing', 'Wireline_Consumer_Specialist',
       'Repair_Wireline_Residential', 'Repair_Support-Chat',
       'Wireline_Consumer_Sales', 'Cable_Seniors_CCO_Support',
       'Wireline_Seniors_CCO_Support', 'COAST', 'TDS_Corporate',
       'Repair_Seniors'], dtype=object)